#### **Librerías a utilizar**

In [117]:
import pandas as pd
from sklearn.ensemble import IsolationForest

#### **Carga de bdd**

In [118]:
columns=[
    "CIUDAD",
    "T00",
    "T07",
    "T08",
    "T10",
    "T11",
    "T12A1",
    "T13",
    "T14",
    "T1701",
    "T1702",
    "T1703",
    "T1705",
    "T1706",
    "T20A1",
    "T20A2",
    "T21A1",
    "T21A2",
    "T22"]
base_turismo = pd.read_spss(
    r"..\bdd\bdd_original\turismodetalleviajes0310.sav", usecols=columns)

#### **Estandarización de nombres de columnas**

In [119]:
new_columns = [
    "dpa",
    "num_viaje",
    "mes_viaje",
    "viaje_realizado_feriado",
    "principal_motivo_viaje",
    "destino_principal_viaje",
    "tipo_transporte_viaje",
    "num_noches_durmieron",
    "tipo_alojamiento",
    "gasto_alojamiento",
    "gasto_alimentacion",
    "gasto_transporte",
    "gasto_visitas",
    "gasto_discotecas",
    "principal_actividad1",
    "principal_actividad2",
    "medio_inf_destino1",
    "medio_inf_destino2",
    "metodo_financiacion"  
]
base_turismo = base_turismo.rename(columns=dict(zip(columns, new_columns)))

#### **Bdd dpa para traer el nombre de la ciudad**

In [120]:
# https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjg47f36fmPAxXWSzABHaitATIQFnoECB0QAQ&url=https%3A%2F%2Fwww.ecuadorencifras.gob.ec%2Fdocumentos%2Fweb-inec%2FPoblacion_y_Demografia%2FCPV_aplicativos%2Fmodulo_cpv%2Fdpa_.xls&usg=AOvVaw0EXZ10-qUS-dCWmAqLLXHM&opi=89978449
dpa = pd.read_excel(
    r"..\bdd\bdd_original\dpa.xlsx")

dpa['ciudad'] = dpa['ciudad'].str.title().str.strip()

In [121]:
base_turismo = pd.merge(
    base_turismo,
    dpa,
    on="dpa",
    how="left"
)

base_turismo = base_turismo.drop(columns=["dpa"], errors='ignore')

base_turismo = base_turismo.rename(columns={"destino_principal_viaje": "dpa"})
base_turismo = pd.merge(
    base_turismo,
    dpa,
    on="dpa",
    how="left"
)

base_turismo = base_turismo.drop(columns=["dpa"], errors='ignore')

base_turismo = base_turismo.rename(columns={"ciudad_x": "ciudad", "ciudad_y": "destino_principal_viaje"})
base_turismo

,num_viaje,mes_viaje,viaje_realizado_feriado,principal_motivo_viaje,tipo_transporte_viaje,num_noches_durmieron,tipo_alojamiento,gasto_alojamiento,gasto_alimentacion,gasto_transporte,gasto_visitas,gasto_discotecas,principal_actividad1,principal_actividad2,medio_inf_destino1,medio_inf_destino2,metodo_financiacion,ciudad,destino_principal_viaje
0,1.0,Febrero,Si,"Diversión, recreación",Vehículo propio,0.0,NaN,0.0,20.0,0.0,0.0,0.0,"Visita a festivales, ferias",Ninguno,Experiencia por visita anterior,Ninguno,Recursos propios,Cuenca,Gualaceo
1,1.0,Febrero,Si,Visitar amigos y/o parientes,Vehículo alquilado,0.0,NaN,0.0,50.0,80.0,0.0,0.0,Otras,Ninguno,Consejo / invitación de amigos o familiares,Ninguno,Recursos propios,Cuenca,Oña
2,1.0,Diciembre,Si,"Diversión, recreación",Vehículo propio,0.0,NaN,0.0,100.0,0.0,0.0,0.0,Visita a playas,Ninguno,Experiencia por visita anterior,Ninguno,Recursos propios,Cuenca,Playas
3,1.0,Enero,No,Gastronomía,Vehículo propio,0.0,NaN,0.0,20.0,0.0,0.0,0.0,Otras,Ninguno,Ninguno,Ninguno,Recursos propios,Cuenca,Gualaceo
4,1.0,Enero,No,Visitar amigos y/o parientes,Vehículo propio,2.0,Vivienda de familiares o amigos,0.0,60.0,0.0,0.0,0.0,Visita a atractivos naturales,Ninguno,Ninguno,Ninguno,Recursos propios,Cuenca,Machala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3453,1.0,Febrero,No,Visitar amigos y/o parientes,Vehículo propio,5.0,Vivienda de familiares o amigos,0.0,10.0,10.0,60.0,0.0,Otras,Ninguno,Experiencia por visita anterior,Ninguno,Recursos propios,La Concordia,Chone
3454,1.0,Febrero,Si,Otros motivos,Vehículo propio,2.0,Vivienda de familiares o amigos,0.0,1.0,4.0,0.0,0.0,Otras,Ninguno,Otros,Ninguno,Recursos propios,La Concordia,Santo Domingo
3455,1.0,Febrero,No,Visitar amigos y/o parientes,Vehículo propio,4.0,Vivienda de familiares o amigos,0.0,65.0,0.0,40.0,20.0,Visita a atractivos naturales,Visita a balnearios,Consejo / invitación de amigos o familiares,Experiencia por visita anterior,Recursos propios,Zonas No Delimitadas,Portoviejo
3456,1.0,Febrero,No,Tratamientos de salud,Autobús,5.0,"Hotel, hostal y similares",60.0,70.0,20.0,0.0,0.0,Otras,Ninguno,Consejo / invitación de amigos o familiares,Ninguno,Recursos propios,Zonas No Delimitadas,Guayaquil


#### **Limpieza de la bdd**

In [122]:
def clean_numeric(col):
    """Convierte columna a numérica reemplazando 'No informa' por 0 sin warnings."""
    col = col.astype(str)
    col = col.replace(["No informa", "nan", "NaN", ""], 0)
    return pd.to_numeric(col, errors='coerce')


def clean_categorical(col):
    """Convierte a texto y rellena NaN por 'No informa'."""
    col = col.astype(str)
    return col.replace(["nan", "NaN", ""], "No informa")

numeric_cols = [
    "num_noches_durmieron",
    "gasto_alojamiento",
    "gasto_alimentacion",
    "gasto_transporte",
    "gasto_visitas",
    "gasto_discotecas",
]

for col in numeric_cols:
    base_turismo[col] = clean_numeric(base_turismo[col])

categorical_cols = [
    "tipo_alojamiento"
]

for col in categorical_cols:
    base_turismo[col] = clean_categorical(base_turismo[col])

base_turismo = base_turismo[~base_turismo.isin([999]).any(axis=1)]

#### **Eliminación de registros atípicos**

In [123]:
numeric_cols = [
    "num_viaje",
    "num_noches_durmieron",
    "gasto_alojamiento",
    "gasto_alimentacion",
    "gasto_transporte",
    "gasto_visitas",
    "gasto_discotecas",
]

X = base_turismo[numeric_cols]

iso = IsolationForest(contamination=0.05, random_state=42)

yhat = iso.fit_predict(X)

mask = yhat != -1
base_turismo = base_turismo[mask]
base_turismo = base_turismo[~base_turismo.isin([10100]).any(axis=1)]
base_turismo

,num_viaje,mes_viaje,viaje_realizado_feriado,principal_motivo_viaje,tipo_transporte_viaje,num_noches_durmieron,tipo_alojamiento,gasto_alojamiento,gasto_alimentacion,gasto_transporte,gasto_visitas,gasto_discotecas,principal_actividad1,principal_actividad2,medio_inf_destino1,medio_inf_destino2,metodo_financiacion,ciudad,destino_principal_viaje
0,1.0,Febrero,Si,"Diversión, recreación",Vehículo propio,0.0,No informa,0.0,20.0,0.0,0.0,0.0,"Visita a festivales, ferias",Ninguno,Experiencia por visita anterior,Ninguno,Recursos propios,Cuenca,Gualaceo
1,1.0,Febrero,Si,Visitar amigos y/o parientes,Vehículo alquilado,0.0,No informa,0.0,50.0,80.0,0.0,0.0,Otras,Ninguno,Consejo / invitación de amigos o familiares,Ninguno,Recursos propios,Cuenca,Oña
2,1.0,Diciembre,Si,"Diversión, recreación",Vehículo propio,0.0,No informa,0.0,100.0,0.0,0.0,0.0,Visita a playas,Ninguno,Experiencia por visita anterior,Ninguno,Recursos propios,Cuenca,Playas
3,1.0,Enero,No,Gastronomía,Vehículo propio,0.0,No informa,0.0,20.0,0.0,0.0,0.0,Otras,Ninguno,Ninguno,Ninguno,Recursos propios,Cuenca,Gualaceo
4,1.0,Enero,No,Visitar amigos y/o parientes,Vehículo propio,2.0,Vivienda de familiares o amigos,0.0,60.0,0.0,0.0,0.0,Visita a atractivos naturales,Ninguno,Ninguno,Ninguno,Recursos propios,Cuenca,Machala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451,1.0,Diciembre,Si,"Diversión, recreación",Vehículo propio,2.0,Vivienda de familiares o amigos,0.0,100.0,0.0,0.0,0.0,Ninguno,Ninguno,Consejo / invitación de amigos o familiares,Ninguno,Recursos propios,La Concordia,Rioverde
3452,1.0,Diciembre,Si,"Diversión, recreación",Autobús,0.0,No informa,0.0,50.0,0.0,0.0,0.0,Visita a atractivos naturales,Ninguno,Consejo / invitación de amigos o familiares,Ninguno,Recursos propios,La Concordia,Atacames
3454,1.0,Febrero,Si,Otros motivos,Vehículo propio,2.0,Vivienda de familiares o amigos,0.0,1.0,4.0,0.0,0.0,Otras,Ninguno,Otros,Ninguno,Recursos propios,La Concordia,Santo Domingo
3456,1.0,Febrero,No,Tratamientos de salud,Autobús,5.0,"Hotel, hostal y similares",60.0,70.0,20.0,0.0,0.0,Otras,Ninguno,Consejo / invitación de amigos o familiares,Ninguno,Recursos propios,Zonas No Delimitadas,Guayaquil


#### **Selección de columnas importantes para le modelo**

In [124]:
# Seleccionar columnas relevantes para el modelo
select_cols = [
    "num_noches_durmieron",
    "gasto_alojamiento", 
    "gasto_alimentacion",
    "gasto_transporte", 
    "gasto_visitas",
    "gasto_discotecas",
    "mes_viaje",
    "principal_actividad1",
    "destino_principal_viaje"]

# Sumar las columnas de gasto para crear gasto_tot
cols_gasto = ["gasto_alojamiento", "gasto_alimentacion", "gasto_transporte", "gasto_visitas", "gasto_discotecas"]
base_turismo["gasto_tot"] = base_turismo[cols_gasto].sum(axis=1)

# Seleccionar solo las columnas deseadas
base_turismo = base_turismo[select_cols + ["gasto_tot"]]

#### **Excluir las categorías de otros o ninguna en principal_actividad1**

In [125]:
base_turismo = base_turismo[~base_turismo["principal_actividad1"].isin(["Otras", "Ninguno"])]


#### **Guardar bdd**

In [126]:
base_turismo.to_excel(r"..\bdd\bdd_limpia\base_turismo_clean.xlsx", index=False)